In [11]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
import datetime
from ipywidgets import interactive
from IPython.display import display, Javascript

import warnings
warnings.filterwarnings('ignore')


parquet_file = r'/home/maanan/sevencommodities/random_deals.parq'

df = pd.read_parquet(parquet_file, engine='auto')

In [12]:
# Keep the date, remove the time

df['executed_date'] = pd.to_datetime(df['executed_date'], format = '%Y-%m-%d').dt.date
df['first_delivery_date'] = pd.to_datetime(df['first_delivery_date'], format = '%Y-%m-%d').dt.date
df['last_delivery_date'] = pd.to_datetime(df['last_delivery_date'], format = '%Y-%m-%d').dt.date
df['last_trading_date'] = pd.to_datetime(df['last_trading_date'], format = '%Y-%m-%d').dt.date

In [13]:
# Create widgets
## Book widget

books = pd.Categorical(df['book'])
books = books.categories

books_dropdown = widgets.Dropdown(
    options=books,
    value=books[0],
    description='Book:',
    disabled=False,
)


## Counterparty widget

counterparty = pd.Categorical(df['counterparty'])
counterparty = counterparty.categories

counter_dropdown = widgets.Dropdown(
    options=counterparty,
    value=counterparty[0],
    description='Counterparty:',
    disabled=False,
)


# Time widget

date_picker = widgets.DatePicker(
    description='Pick a Date',
    disabled=False,
)
date_picker.add_class("start-date")

script = Javascript("\
                const query = '.start-date > input:first-of-type'; \
                document.querySelector(query).setAttribute('min', '2020-12-01'); \
                document.querySelector(query).setAttribute('max', '2025-01-01'); \
        ")


In [14]:
# This function will allow us to do the cascading:
# Once a date has been selected, all the deals whos LTD was before that date will be cascaded as follow
# Every Quarter will be cascaded into three Months
# Every Year will be cascaded into three months and three quarters
# Months are not to be cascaded

def split_tenor(row):
    start, year = row['new_window'].split(" ")
    if start == "Cal":
        months = ["Jan", "Feb", "Mar", "Q2", "Q3", "Q4"]
        year = int(year) + 1
    elif start == "Q1":
        months = ["Jan", "Feb", "Mar"]
    elif start == "Q2":
        months = ["Apr", "May", "Jun"]
    elif start == "Q3":
        months = ["Jul", "Aug", "Sep"]
    elif start == "Q4":
        months = ["Oct", "Nov", "Dec"]
    else:
        return row['new_window'], row['volume']

    if start == "Cal":
        split_vol = [row['volume']/12] * 3 + [row['volume']/4] * 3
    else:
        split_vol = [row['volume']/len(months)] * len(months)
    
    return [f"{m} {year}" for m in months], split_vol


In [15]:
# This function will aggregate the shorter periods into longer periods

# Add cocatenation for year



def ConvtoQuarter(tenor, delivery):
  if tenor == "quarter":
    if delivery[:3] in ['Jan', 'Feb', 'Mar']:
      return "Q1 "+delivery[-2:]
    elif delivery[:3] in ['Apr', 'May', 'Jun']:
      return "Q2 "+delivery[-2:]
    elif delivery[:3] in ['Jul', 'Aug', 'Sep']:
      return "Q3 " + delivery[-2:]
    elif delivery[:3] in ['Oct', 'Nov', 'Dec']:
      return "Q4 " + delivery[-2:]
  else:
      return delivery

In [16]:
def filter_function(bookcode, cpartycode, datecode):
                
    filtered = df[(df['book'] == bookcode) & (df['counterparty'] == cpartycode)]
        
    filtered = filtered.drop(['deal_id','book','counterparty','strategy','commodity_code','trading_unit'], axis=1)
            
    filtered.loc[filtered['buy_sell'] == 'sell', 'volume'] = -filtered['volume']
        
    filtered['new_window'] = filtered.apply(lambda x: ConvtoQuarter(x['tenor'], x['delivery_window']), axis=1)
            
    split_1 = filtered[filtered['last_trading_date'] < datecode].copy()
    
    split_2 = filtered[filtered['last_trading_date'] >= datecode].copy()
        
    split_1["new_window"], split_1["new_volume"] = zip(*split_1[["volume", "new_window"]].apply(split_tenor, axis = 1))
            
    split_1 = split_1.explode(["new_window", "new_volume"])
    
    split_2['new_volume'] = split_2['volume']
        
    filtered = pd.concat([split_1, split_2], ignore_index=True)
            
    filtered = pd.pivot_table(filtered, values="new_volume", index=["new_window"], columns=["commodity_name"], aggfunc=np.sum)
    
    with report_output:
        report_output.clear_output()
        display(filtered.round(1))  
        
w = interactive(filter_function, bookcode=books_dropdown, cpartycode=counter_dropdown, datecode=date_picker) 

display(w)
report_output = widgets.Output()
display(report_output)

interactive(children=(Dropdown(description='Book:', options=('Book_1', 'Book_2', 'Book_3', 'Book_4', 'Book_5',…

Output()